In [ ]:
%load_ext autoreload
%autoreload 2
import os, pickle
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch
%matplotlib inline
%config InlineBackend.figure_format='retina'

import warnings
warnings.filterwarnings("ignore")
import models.utils as utils
import csv

from torchvision import transforms
import evals.embed_evals as evals
import pickle
import utils.plot_data as plot
import shutil
import utils.gen_triplets as gen
import pandas as pd
from omegaconf import OmegaConf as oc

In [61]:
a = os.listdir(dir)
a.sort()
a

['test_triplets.pkl',
 'test_triplets_filtered.pkl',
 'train_triplets.pkl',
 'train_triplets_filtered.pkl',
 'valid_triplets.pkl',
 'valid_triplets_filtered.pkl']

In [85]:
for align in  [0.7, 0.775,0.8,0.85,0.9,0.95,0.975,1]:
    dir = f"align={str(align)}"
    print(dir)
    dir = os.path.join("data/wv_2d_triplets",dir)
    files = os.listdir(dir)
    files.sort()
    for f in files:
        f = os.path.join(dir,f)
        print(len(pickle.load(open(f,"rb"))))

align=0.7
10000
9159
8000
7474
2000
1775
align=0.775
10000
8466
8000
6969
2000
1742
align=0.8
10000
8499
8000
6986
2000
1746
align=0.85
10000
8485
8000
6977
2000
1755
align=0.9
10000
8441
8000
6954
2000
1753
align=0.95
10000
8633
8000
7086
2000
1786
align=0.975
10000
9332
8000
7553
2000
1868
align=1
10000
9381
8000
7617
2000
1877


In [ ]:
train_triplets = pickle.load(open("data/wv_2d_triplets/align=0.7/train_triplets.pkl", "rb"))
valid_triplets = pickle.load(open("data/wv_2d_triplets/align=0.7/valid_triplets.pkl", "rb"))
test_triplets = pickle.load(open("data/wv_2d_triplets/align=0.7/test_triplets.pkl", "rb"))

In [ ]:

x_train = pickle.load(open("embeds/wv_2d/train.pkl", "rb"))
x_test = pickle.load(open("embeds/wv_2d/test.pkl", "rb"))

In [ ]:
w2s = np.arange(0,1,0.0001)
y = [evals.distorted_1nn(x_train, y_train, x_test, y_test, [1,w2],[2,2]) for w2 in w2s]
plt.plot(w2s,y)

In [ ]:
w2=0.5
evals.distorted_1nn(x_train, y_train, x_test, y_test, [1,w2],[2,2]) 

In [ ]:
np.unique(y)

In [ ]:
y

In [ ]:
w2=0.5
evals.distorted_1nn(x_train, y_train, x_test, y_test, [1,w2],[2,2])

In [ ]:
evals.distorted_1nn(x_train, y_train, x_test, y_test, [1,-2],[2,2])

In [ ]:
for align in [0.775, 0.8, 0.85, 0.9, 0.95, 0.975, 1]:
    path = os.path.join("data/wv_2d_triplets",f"align={str(align)}")
    os.mkdir(path)

### vis results

In [ ]:
results = pd.read_csv("models/results.csv")
cols = ["test_triplet_acc", "test_1nn_acc", "decision_support"]
TN_df = results.loc[results["wandb_group"]=="TN_test"]
TN_df

In [ ]:
plt.plot(np.arange(9), TN_df["test_triplet_acc"][::-1])

In [ ]:
results = pd.read_csv("models/results.csv")
cols = ["test_triplet_acc", "test_1nn_acc", "decision_support"]
MTL_df = results.loc[results["wandb_group"]=="MTL0.8"]
MTL_df

In [ ]:
RESN_df = pd.read_csv("/net/scratch/tianh/explain_teach/models/results_csv/new_resn/wv_RESN.csv")
TN_df = pd.read_csv("/net/scratch/tianh/explain_teach/models/results_csv/new_resn/wv_TN.csv")
MTL_df = pd.read_csv("/net/scratch/tianh/explain_teach/models/results_csv/new_resn/wv_MTL.csv")
plots = ["test_triplet_acc", "test_1nn_acc", "decision_support"]
lw=3
for plot in plots:
    plt.figure(figsize=(8, 6))
    plt.plot(RESN_df["align"], RESN_df[plot], lw=lw)
    plt.plot(TN_df["align"], TN_df[plot], lw=lw)
    plt.plot(MTL_df["align"], MTL_df[plot], lw=lw)
    plt.title(plot)
    plt.legend(["RESN", "TN", "MTL"])
    plt.xticks(RESN_df["align"])
    plt.xlabel("alignment")
    plt.ylim((0,1))
    plt.savefig(f"figs/wv{plot}.pdf")
    plt.show()

In [ ]:
test_triplets = []
for triplet in stt:
    test, t1, t2 = triplet
    triplet = [test_img2idx[test], train_img2idx[t1], train_img2idx[t2]]
    test_triplets.append(triplet)

In [ ]:
pickle.dump(test_triplets, open("/net/scratch/tianh/explain_teach/data/bm_triplets/test_triplets.pkl","wb"))

In [ ]:
test_f = utils.dataset_filenames("/net/scratch/tianh-shared/bm/test")[:,0]
test_img2idx = {x.split("/")[-1]:i for i,x in enumerate(test_f)}
pickle.dump(test_img2idx, open("/net/scratch/tianh/explain_teach/data/img2idx/bm_test.pkl","wb"))

In [ ]:

df = pd.DataFrame(data={"wandb_group": [0], "wandb_name": [0],
    "test_clf_acc": [0], "test_clf_loss": [0], "test_1nn_acc": [0], "test_triplet_acc":[0], "decision_support": [0]})
df.to_csv("models/results.csv", index=False)

### triplet gen

In [80]:
train_embeds = pickle.load(open("/net/scratch/tianh/explain_teach/embeds/wv_2d/train.pkl","rb"))
valid_embeds = pickle.load(open("/net/scratch/tianh/explain_teach/embeds/wv_2d/valid.pkl","rb"))
test_embeds = pickle.load(open("/net/scratch/tianh/explain_teach/embeds/wv_2d/test.pkl","rb"))
len_train = len(train_embeds)
len_valid = len(valid_embeds)
len_test = len(test_embeds)
y_train = np.array([x[1] for x in torchvision.datasets.ImageFolder("data/weevil_vespula/train")])
y_valid = np.array([x[1] for x in torchvision.datasets.ImageFolder("data/weevil_vespula/valid")])
y_test = np.array([x[1] for x in torchvision.datasets.ImageFolder("data/weevil_vespula/test")])
classes = [0,1]
y_train_idx = [np.where(y_train==c)[0] for c in classes]
y_valid_idx = [np.where(y_valid==c)[0] for c in classes]
y_test_idx = [np.where(y_test==c)[0] for c in classes]

In [83]:
w2s = [0.0001,0.001,0.0025,0.005,0.05,0.5,1]
aligns = [0.775,0.8,0.85,0.9,0.95,0.975,1]

In [84]:
for w2,align in zip(w2s, aligns):
    dir = f"align={str(align)}"
    weights = [1,w2]
    total = 10000
    train_triplets = gen.sample_triplets(np.arange(len_train), int(total*0.8), weights, train_embeds, [2,2])
    valid_triplets = gen.sample_mixed_triplets(np.arange(len_valid),np.arange(len_train), int(total*0.2), weights, valid_embeds, train_embeds, [2,2])
    test_triplets = gen.sample_mixed_triplets(np.arange(len_test),np.arange(len_train), total, weights, test_embeds, train_embeds, [2,2])
    pickle.dump(train_triplets, open(f"/net/scratch/tianh/explain_teach/data/wv_2d_triplets/{dir}/train_triplets.pkl", "wb"))
    pickle.dump(valid_triplets, open(f"/net/scratch/tianh/explain_teach/data/wv_2d_triplets/{dir}/valid_triplets.pkl", "wb"))
    pickle.dump(test_triplets, open(f"/net/scratch/tianh/explain_teach/data/wv_2d_triplets/{dir}/test_triplets.pkl", "wb"))
    train_triplets_filtered = []
    valid_triplets_filtered = []
    test_triplets_filtered = []
    for t in train_triplets:
        a,p,n = t
        if not ((a in y_train_idx[0] and p in y_train_idx[1] and n in y_train_idx[0]) or (a in y_train_idx[1] and p in y_train_idx[0] and n in y_train_idx[1])): train_triplets_filtered.append(t)
    for t in valid_triplets:
        a,p,n = t
        if not ((a in y_valid_idx[0] and p in y_train_idx[1] and n in y_train_idx[0]) or (a in y_valid_idx[1] and p in y_train_idx[0] and n in y_train_idx[1])): valid_triplets_filtered.append(t)
    for t in test_triplets:
        a,p,n = t
        if not ((a in y_test_idx[0] and p in y_train_idx[1] and n in y_train_idx[0]) or (a in y_test_idx[1] and p in y_train_idx[0] and n in y_train_idx[1])): test_triplets_filtered.append(t)
    print(len(train_triplets_filtered), len(valid_triplets_filtered), len(test_triplets_filtered))
    pickle.dump(train_triplets_filtered, open(f"data/wv_2d_triplets/{dir}/train_triplets_filtered.pkl","wb"))
    pickle.dump(valid_triplets_filtered, open(f"data/wv_2d_triplets/{dir}/valid_triplets_filtered.pkl","wb"))
    pickle.dump(test_triplets_filtered, open(f"data/wv_2d_triplets/{dir}/test_triplets_filtered.pkl","wb"))

6969 1742 8466
6986 1746 8499
6977 1755 8485
6954 1753 8441
7086 1786 8633
7553 1868 9332
7617 1877 9381


In [69]:
t

[26, 70, 52]

In [48]:
base = oc.load("models/configs/wv_2d/triplets/align=0.7.yaml")
base_filtered = oc.load("models/configs/wv_2d/triplets/align=0.7_filtered.yaml")

In [ ]:
for w2,align in zip(w2s, aligns):
    weights = [1,w2]
    train_triplets = base.train_triplets.replace("0.7",str(align))
    valid_triplets = base.valid_triplets.replace("0.7",str(align))
    test_triplets = base.test_triplets.replace("0.7",str(align))
    wandb_group = base.wandb_group.replace("0.7",str(align))
    overwrite = {"weights":weights, "train_triplets":train_triplets,"valid_triplets":valid_triplets,
    "test_triplets":test_triplets,"wandb_group":wandb_group}
    oc.save(oc.merge(base,overwrite), f"models/configs/wv_2d/triplets/align={align}.yaml")

In [54]:
for w2,align in zip(w2s, aligns):
    weights = [1,w2]
    train_triplets = base_filtered.train_triplets.replace("0.7",str(align))
    valid_triplets = base_filtered.valid_triplets.replace("0.7",str(align))
    test_triplets = base_filtered.test_triplets.replace("0.7",str(align))
    wandb_group = base_filtered.wandb_group.replace("0.7",str(align))
    overwrite = {"weights":weights, "train_triplets":train_triplets,"valid_triplets":valid_triplets,
    "test_triplets":test_triplets,"wandb_group":wandb_group}
    oc.save(oc.merge(base_filtered,overwrite), f"models/configs/wv_2d/triplets/align={align}_filtered.yaml")

## cross validation

In [ ]:
c1, c2, c3, c4 = np.arange(1583), np.arange(1583,3166), np.arange(600,900), np.arange(900,1200)
splits = utils.cross_val_multiclass([c1,c2], 10)

In [ ]:
c1 = np.arange(1583)
valid_test = np.random.choice(c1, 300,replace=False)
c1_valid = np.random.choice(valid_test, 150, replace=False)
c1_test = np.setdiff1d(valid_test, c1_valid)
c1_train = np.setdiff1d(c1, valid_test)

In [ ]:
c2 = np.arange(1583,5856)
valid_test = np.random.choice(c2, 300,replace=False)
c2_valid = np.random.choice(valid_test, 150, replace=False)
c2_test = np.setdiff1d(valid_test, c2_valid)
c2_train = np.setdiff1d(c2, valid_test)

In [ ]:
pickle.dump(splits,open("models/pneumonia_splits_imb.pkl","wb"))

In [ ]:
src_dir = "/net/scratch/tianh-shared/warblers/inat/4class"
dst_dir = "/net/scratch/tianh-shared/warblers/data"
split5 = pickle.load(open("models/bird_splits.pkl","rb"))[5]

In [ ]:
train, val, test = split5
instances = utils.dataset_filenames(src_dir)

In [ ]:
labels = [v[1] for v in instances[train]]
unique = set(labels)
counts = [labels.count(c) for c in unique]
counts

In [ ]:
lambdas = np.arange(0.1, 1, 0.1)
clf = [0.875, 0.925, 0.95, 0.975, 0.975, 0.975, 1, 1, 0.975]
triplet = [0.897,0.913,0.913,0.855,0.841,0.826,0.841,0.841,0.841]

In [ ]:
lw=4
plt.rc('legend', fontsize=15)
plt.figure(figsize=(10,6))
plt.axhline(1, c='black',linewidth=lw, label="ResNET clf acc")
plt.plot(lambdas, clf,linewidth=lw, label="MTL clf acc")
plt.plot(lambdas, triplet,linewidth=lw, label="MTL triplet acc")
plt.axhline(0.841, c='green',linewidth=lw, label="TN triplet acc")
plt.xlabel('lambda')
# plt.title("MTL trained on human triplets")
plt.legend(loc='upper right', bbox_to_anchor=(1.1, -0.1),fancybox=True, shadow=True, ncol=4)
plt.savefig("figs/MTL_lambdas.pdf", format="pdf", bbox_inches="tight")

In [ ]:

triplets = pickle.load(open("/net/scratch/tianh/triplet-webapp/backup/trial0/butterfly.triplets.pkl","rb"))

In [ ]:
train_idx = pickle.load(open("data/bm_train_idx.pkl", "rb"))[1]
val_idx  = pickle.load(open("data/bm_valid_idx.pkl", "rb"))[1]

In [ ]:
train_triplets = []
val_triplets = []
for triplet in triplets:
    a,p,n = triplet
    if a in train_idx.keys() and p in train_idx.keys() and n in train_idx.keys():
        new = [train_idx[a], train_idx[p], train_idx[n]]
        train_triplets.append(new)
    elif a in val_idx.keys() and p in val_idx.keys() and n in val_idx.keys():
        new = [val_idx[a], val_idx[p], val_idx[n]]
        val_triplets.append(new)

In [ ]:

train_triplets = "/net/scratch/tianh/explain_teach/data/bm_triplets/3c2_unique=182/train_triplets.pkl"
valid_triplets = "/net/scratch/tianh/explain_teach/data/bm_triplets/3c2_unique=182/valid_triplets.pkl"
train_triplets = pickle.load(open(train_triplets, "rb"))
valid_triplets = pickle.load(open(valid_triplets, "rb"))

In [ ]:

train_lpips_distance = "/net/scratch/tianh/explain_teach/embeds/lpips/lpips.bm.train.pkl"
valid_lpips_distance = "/net/scratch/tianh/explain_teach/embeds/lpips/lpips.bm.valid.pkl"
train_lpips_distance = pickle.load(open(train_lpips_distance, "rb"))
valid_lpips_distance = pickle.load(open(valid_lpips_distance, "rb"))

In [ ]:
correct = 0
total = 0
for htriplet in train_triplets:
    total += 1
    a,p,n = htriplet[0], htriplet[1], htriplet[2]
    if train_lpips_distance[a, p] < train_lpips_distance[a, n]:
        correct += 1
correct/total


In [ ]:
correct = 0
total = 0
for htriplet in valid_triplets:
    total += 1
    a,p,n = htriplet[0], htriplet[1], htriplet[2]
    if valid_lpips_distance[a, p] < valid_lpips_distance[a, n]:
        correct += 1
correct/total


In [ ]:
# train_pairwise_distance= "embeds/lpips.bm.train.pkl" 
# valid_pairwise_distance= "embeds/lpips.bm.valid.pkl" 
# train_pairwise_distance = pickle.load(open(train_pairwise_distance, "rb"))
# valid_pairwise_distance = pickle.load(open(valid_pairwise_distance, "rb"))

train_triplets = "/net/scratch/tianh/explain_teach/data/bm_triplets/train_triplets.pkl"
valid_triplets = "/net/scratch/tianh/explain_teach/data/bm_triplets/valid_triplets.pkl"
train_triplets = pickle.load(open(train_triplets, "rb"))
valid_triplets = pickle.load(open(valid_triplets, "rb"))

In [ ]:
train = pickle.load(open("embeds/TN_bm_train.pkl","rb"))
valid = pickle.load(open("embeds/TN_bm_valid.pkl","rb"))
triplets = pickle.load(open("data/bm_triplets/val2train_triplets.pkl","rb"))

In [ ]:
acc = evals.val2train_triplet_acc(train, valid, triplets)
acc

In [ ]:
acc = evals.triplet_acc(train, train_triplets)
acc

In [ ]:
acc = evals.triplet_acc(valid, valid_triplets)
acc

In [ ]:

train_triplets = "/net/scratch/tianh/bm/triplets/train_triplets.pkl"
valid_triplets = "/net/scratch/tianh/bm/triplets/valid_triplets.pkl"
# clf_train_triplets = "/net/scratch/tianh/bm/triplets/clf_train_triplets.pkl"
# clf_valid_triplets = "/net/scratch/tianh/bm/triplets/clf_valid_triplets.pkl"

train_triplets = pickle.load(open(train_triplets, "rb"))

In [ ]:

train_dir = "/net/scratch/hanliu-shared/data/bm/train"
valid_dir = "/net/scratch/hanliu-shared/data/bm/valid"
train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=utils.bm_transform())
valid_dataset = torchvision.datasets.ImageFolder(valid_dir, transform=utils.bm_transform())
train_inputs = torch.tensor(np.array([data[0].numpy() for data in train_dataset]))
valid_inputs = torch.tensor(np.array([data[0].numpy() for data in valid_dataset]))
train_labels = torch.tensor(np.array([data[1] for data in train_dataset]))
valid_labels = torch.tensor(np.array([data[1] for data in valid_dataset]))

In [ ]:
np.where(train_labels==0)

In [ ]:
a = pickle.load(open("/net/scratch/tianh/bm/triplets/clf_train_triplets.pkl","rb"))

In [ ]:
idx = np.random.choice(len(combs), 32, replace=False)
sample = combs[idx]
len(np.unique(sample.flatten()))

In [ ]:
import torch
a = torch.tensor(sample)

In [ ]:
torch.unique(torch.flatten(a))

In [ ]:
triplet_idx = []
for c in combs:
    x1, x2, x3 = c[0], c[1], c[2]
    if train_pairwise_distance[x1, x2] > train_pairwise_distance[x1, x3]:
        triplet_idx.append([x1, x3, x2])
    else:
        triplet_idx.append([x1, x2, x3])

In [ ]:
pickle.dump(np.array(triplet_idx), open('/net/scratch/tianh/bm/triplets/train_triplets.pkl',"wb"))

In [ ]:
a = pickle.load(open('/net/scratch/tianh/bm/triplets/train_triplets.pkl',"rb"))
a.shape

In [ ]:
np.array(triplet_idx).shape

In [ ]:
food100 = []
with open('/net/scratch/tianh/food100-dataset/all-triplets.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        food100.append(row)

In [ ]:

transform = transforms.Compose([
    transforms.Resize([230,230]),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
data_dir = '/net/scratch/tianh/food100-dataset/images'
dataset = torchvision.datasets.ImageFolder(data_dir, transform=transform)
total_idx = np.arange(len(dataset))

In [ ]:
train_img_idx = np.random.choice(total_idx, int(len(total_idx)*0.615), replace=False)
valid_img_idx = np.setdiff1d(total_idx, train_img_idx)

train_data = torch.tensor(np.array([dataset[i][0].numpy() for i in train_img_idx]))
valid_data = torch.tensor(np.array([dataset[i][0].numpy() for i in valid_img_idx]))
train_label = torch.tensor(np.array([dataset[i][1] for i in train_img_idx]))
valid_label = torch.tensor(np.array([dataset[i][1] for i in valid_img_idx]))

valid_triplets = []
for t in triplets:
    if t[0] in valid_label and t[1] in valid_label and t[2] in valid_label:
        valid_triplets.append(t)
train_triplets = []
for t in triplets:
    if t[0] in train_label and t[1] in train_label and t[2] in train_label:
        train_triplets.append(t)
len(train_triplets), len(valid_triplets)

In [ ]:
41685/11018

In [ ]:
len(triplets)*0.2

In [ ]:
triplets = pickle.load(open("/net/scratch/tianh/food100-dataset/triplets_idx.pkl", "rb"))

In [ ]:
train_triplets

In [ ]:
total_idx = np.arange(100)
train_idx = np.random.choice(total_idx, len(total_idx)*8//10, replace=False)
valid_idx = np.setdiff1d(total_idx, train_idx)
train_triplets = 0
valid_triplets = 0
for t in triplets:
    if t[0] in train_idx:
        train_triplets += 1
    else:
        valid_triplets += 1
train_triplets/len(triplets), valid_triplets/len(triplets)

In [ ]:
path = "/net/scratch/tianh/food100-dataset/images"
dir_list = os.listdir(path)
dir_list

In [ ]:
import shutil
for image in dir_list:
    image_dir = os.path.join(path, image[:-4]) 
    if not os.path.exists(image_dir):
        os.mkdir(image_dir) 
        shutil.copy(os.path.join(path,image),image_dir)

### evaluating lpips(bm) by class

In [ ]:
scores = []
for i in range(80):
    correct = 0
    total = 0
    for j in range(80):
        for k in range(80,160):
            total += 1
            if train_dist[i,j] <= train_dist[i,k]:
                correct += 1
    scores.append(correct/total)
    
for i in range(80, 160):
    correct = 0
    total = 0
    for j in range(80):
        for k in range(80,160):
            total += 1
            if train_dist[i,j] >=  train_dist[i,k]:
                correct += 1
    scores.append(correct/total)
sum(scores)/len(scores)

## evaluating lpips(food) w/ triplets

In [ ]:
distances_matrix = pickle.load(open("lpips.food.pkl", "rb"))
triplets = pickle.load(open("/net/scratch/tianh/food100-dataset/triplets_idx.pkl", "rb"))

In [ ]:
total = len(triplets)
correct = 0
for triplet in triplets:
    if distances_matrix[triplet[0], triplet[1]] <= distances_matrix[triplet[0], triplet[2]]:
        correct += 1
correct/total

## triplet generalization error

In [ ]:
import os, pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances 
import torch
import torchvision
from tqdm import tqdm
pdist = torch.nn.PairwiseDistance()
%matplotlib inline
%config InlineBackend.figure_format='retina'

np.random.seed(42)
# def euc_dist(x, y): return euclidean_distances([x],[y])[0][0]
def euc_dist(x, y): return np.sqrt(np.dot(x, x) - 2 * np.dot(x, y) + np.dot(y, y))

In [ ]:
def a(f):
    return f(3)
def f(x): return x+1
a(f)

In [ ]:
import tste
triplets = "/net/scratch/tianh/bm/triplets/train_triplets.pkl"
triplets = np.array(pickle.load(open(triplets, "rb")))
embedding = tste.tste(triplets, no_dims=2, verbose=False, max_iter=1000)

In [ ]:
train_align = []
for triplet in tqdm(triplets):
    a, p, n = triplet
    ap = euc_dist(embedding[a], embedding[p]) 
    an = euc_dist(embedding[a], embedding[n])
    train_align.append(ap < an)
print(np.mean(train_align))

In [ ]:
triplets = np.array(pickle.load(open("/net/scratch/tianh/food100-dataset/triplets_idx.pkl", "rb")))
valid_path = "model/embeds/triplet_net_food_triplet.pkl"
X_train = pickle.load(open(valid_path, "rb"))


In [ ]:
train_align = []
for triplet in tqdm(triplets):
    a, p, n = triplet
    ap = euc_dist(X_train[a], X_train[p]) 
    an = euc_dist(X_train[a], X_train[n])
    train_align.append(ap < an)
print(np.mean(train_align))

In [ ]:
X_train.shape

In [ ]:
model = 'triplet_net_food'
name = 'no_linear'
train_path = "/net/scratch/hanliu-shared/data/bm/embs/dwac_train_emb10.merged2.pkl"
valid_path = "/net/scratch/hanliu-shared/data/bm/embs/dwac_valid_emb10.merged2.pkl"
# train_path = '{}/{}_train_{}.pkl'.format("embeds", model, name)
# valid_path = '{}/{}_valid_{}.pkl'.format("embeds", model, name)
X_train = pickle.load(open(train_path, "rb"))
X_train = X_train[3]
X_valid = pickle.load(open(valid_path, "rb"))
X_valid=X_valid[3]
# valid_path = 'model/xd.pkl'
# X_train = pickle.load(open(valid_path, "rb"))

In [ ]:
train_align = []
train_dist = pickle.load(open("embeds/lpips.bm.train.pkl", "rb"))
# train_dist = train_dist[np.array(subset_idx)]
combs = torch.combinations(torch.arange(0, len(train_dist)).int(), r=3)
for c in tqdm(combs):
    a, p, n = c
# for i in tqdm(range(10000)):
#     a, p, n = np.random.choice(len(X_train), 3, replace=False)
    ap = train_dist[a, p] < train_dist[a, n]
    rd = euc_dist(X_train[a], X_train[p]) < euc_dist(X_train[a], X_train[n])
    train_align.append(ap == rd)
print(np.mean(train_align))

In [ ]:
valid_align = []
valid_dist = pickle.load(open("embeds/lpips.bm.valid.pkl", "rb"))
combs = torch.combinations(torch.arange(0, len(valid_dist)-1).int(), r=3)
for c in combs:
    a, p, n = c
    ap = valid_dist[a, p] < valid_dist[a, n]
    rd = euc_dist(X_valid[a], X_valid[p]) < euc_dist(X_valid[a], X_valid[n])
    valid_align.append(ap == rd)
print(np.mean(valid_align))

In [ ]:
d_ap = pdist(torch.tensor(X_valid[combs[:,0]]), torch.tensor(X_valid[combs[:,1]]))
d_an = pdist(torch.tensor(X_valid[combs[:,0]]), torch.tensor(X_valid[combs[:,2]]))

In [ ]:
(d_ap < d_an).float().mean()

## human-compatible example selection

In [ ]:
model, name = 'dwac', 'emb10.merged2'
title = "prostatex"

train_path = 'embeds/{}_train_{}.pkl'.format(model, name)
f_train_dwac, y_train_dwac, x_train_dwac = pickle.load(open(train_path, "rb"))

In [ ]:
_, pdash_examples, _ = pdash(x_train_dwac,x_train_dwac,10,kernelType="Gaussian")
pdash_examples

dwac-lpips

In [ ]:
lpips = pickle.load(open("embeds/lpips.prostatex.train+valid.pkl","rb"))
lpips.shape

In [ ]:
_, examples, _ = pdash_human.pdash(x_train_dwac,x_train_dwac, 10,lpips,kernelType="Gaussian")
examples

In [ ]:
_, examples, _ = pdash_human.pdash(x_train_dwac,x_train_dwac, 10,lpips, f_h_scale=0.1, kernelType="Gaussian")
examples

In [ ]:
_, examples, _ = pdash_human.pdash(x_train_dwac,x_train_dwac, 10,lpips, f_h_scale=1, kernelType="Gaussian")
examples

In [ ]:
utils.vis_proto(x_train_dwac, y_train_dwac, examples, "")

In [ ]:
utils.vis_proto(x_train_dwac, y_train_dwac, pdash_examples, "")

In [ ]:
L = 0.5
Kernel = 'Gaussian'
Gamma = 0.5
k_range = [1, 3]
m_range = list(range(3, 11))

In [ ]:
dataset = 'prostatex'
model = 'dwac'
name = 'emb10.merged2'
train_path = 'embeds/{}_train_{}.pkl'.format(model, name)
valid_path = 'embeds/{}_valid_{}.pkl'.format(model, name)
f_train, y_train, X_train = pickle.load(open(train_path, "rb"))
f_valid, y_valid, X_valid = pickle.load(open(valid_path, "rb"))
data = X_train, y_train, X_valid, y_valid

In [ ]:
dwac_f_scores_knn, dwac_f_score_svm = utils.get_full_score(data, k_range)
dwac_r_means_knn, dwac_r_confs_knn, dwac_r_means_svm, dwac_r_confs_svm = utils.get_random_score(data, k_range, m_range)
dwac_p_scores_knn, dwac_p_scores_svm = utils.get_protodash_score(data, k_range, m_range)

In [ ]:
def get_pdash_score(data, k_range, m_range, f_h_scale):
    p_idss = {}
    X_train, y_train, X_valid, y_valid = data
    for m in m_range:
        if m not in p_idss:
            try:
                _, index, _ = pdash_human.pdash(X_train,X_train, m,lpips, f_h_scale=f_h_scale, kernelType="Gaussian")
            except AttributeError:
                index = [0] * m
                print("error for m={}".format(m))
            p_idss[m] = index
    p_scores_knn, p_scores_svm = [], []
    for k in k_range:
        for m in m_range:
            p_scores_knn.append(utils.get_knn_score(k, data, p_idss[m]))
            try:
                s = utils.get_svm_score(k, data, p_idss[m])
            except:
                s = 0
            p_scores_svm.append(s)
    p_scores_knn = np.array(p_scores_knn).reshape(len(k_range), len(m_range))
    p_scores_svm = np.array(p_scores_svm).reshape(len(k_range), len(m_range))
    return p_scores_knn, p_scores_svm

In [ ]:
pdash_scores_knn1,pdash_scores_svm = get_pdash_score(data, k_range, m_range, 1)
pdash_scores_knn01,pdash_scores_svm = get_pdash_score(data, k_range, m_range, 0.1)
pdash_scores_knn001,pdash_scores_svm = get_pdash_score(data, k_range, m_range, 0.01)
pdash_scores_knn0001,pdash_scores_svm = get_pdash_score(data, k_range, m_range, 0.001)
pdash_scores_knn00001,pdash_scores_svm = get_pdash_score(data, k_range, m_range, 0.00001)

In [ ]:
fig, ax = plt.subplots(1, len(k_range), figsize=(16, 6), sharey=True)
for k in range(len(k_range)):
    ax[k].axhline(dwac_f_scores_knn[k], c='black')
    ax[k].plot(m_range, dwac_r_means_knn[k])
    ax[k].fill_between(m_range, dwac_r_means_knn[k] + dwac_r_confs_knn[k] / 2, dwac_r_means_knn[k] - dwac_r_confs_knn[k] / 2, alpha=0.5)
    ax[k].plot(m_range, dwac_p_scores_knn[k])
    # ax[k].plot(m_range, pdash_scores_knn1[k])
    ax[k].plot(m_range, pdash_scores_knn01[k])
    ax[k].plot(m_range, pdash_scores_knn001[k])
    # ax[k].plot(m_range, pdash_scores_knn0001[k])
    # ax[k].plot(m_range, pdash_scores_knn00001[k])
    ax[k].set_ylim(0.3, 1)
    ax[k].set_xticks(m_range)
    ax[k].set_title('K={}'.format(k_range[k]))
    ax[k].legend(['full', 'random', '', 'protodash', '1NN_constraint', 'pdash_h','0.0001'])
fig.suptitle('{}, {}.{}, AUC, KNN weights '.format(dataset, model, "lpips") + "uniform")

## generate triplets from LPIPS distance matrices
triplets.shape := [k, 3]

triplets[i] = [anchor, positive, negative]

In [ ]:
distance_matrix = pickle.load(open("lpips.prostatex.train+valid.pkl", "rb"))
distance_matrix.shape

In [ ]:
triplets_fname = "data/triplets.px.train+valid.pkl"
tste_fname = "data/tste.px.train+valid.pkl"

In [ ]:
# embeds = utils.get_tste(distance_matrix, triplets_fname, tste_fname, max_iter=100)

In [ ]:
model, name = 'dwac', 'emb10.merged2'
title = "prostatex"

train_path = '{}_train_{}.pkl'.format(model, name)
f_train_dwac, y_train_dwac, x_train_dwac = pickle.load(open(train_path, "rb"))
valid_path = '{}_valid_{}.pkl'.format(model, name)
f_valid_dwac, y_valid_dwac, x_valid_dwac = pickle.load(open(valid_path, "rb"))
vis_data(x_train_dwac, y_train_dwac, x_valid_dwac, y_valid_dwac, title, save=False)
x_train_dwac.shape

In [ ]:
title = "prostatex, tste"
model, name = "tste", "px"

train_path = '{}_train_{}.pkl'.format(model, name)
f_train_tste, y_train_tste, x_train_tste = pickle.load(open(train_path, "rb"))
valid_path = '{}_valid_{}.pkl'.format(model, name)
f_valid_tste, y_valid_tste, x_valid_tste = pickle.load(open(valid_path, "rb"))
vis_data(x_train_tste, y_train_tste, x_valid_tste, y_valid_tste, title, save=False)
x_train_tste.shape

In [ ]:
x_snack = np.load("embeds/snack_dwac+lpips.npy")
y_snack_path = "embeds/tste_all_px.pkl"
y_snack = pickle.load(open(y_snack_path, "rb"))[1]
assert (x_snack.shape[0]==y_snack.shape[0])
vis_data_all(x_snack, y_snack, title, save=False)
x_snack.shape

## knn/svm on tste embeds

In [ ]:
L = 0.5
Kernel = 'Gaussian'
Gamma = 0.5

k_range = [1, 3]
m_range = list(range(3, 11))

In [ ]:
dataset = 'prostatex'
model = 'dwac'
name = 'emb10.merged2'
train_path = '{}_train_{}.pkl'.format(model, name)
valid_path = '{}_valid_{}.pkl'.format(model, name)
f_train, y_train, X_train = pickle.load(open(train_path, "rb"))
f_valid, y_valid, X_valid = pickle.load(open(valid_path, "rb"))
data = X_train, y_train, X_valid, y_valid

In [ ]:
dwac_f_scores_knn, dwac_f_score_svm = utils.get_full_score(data, k_range)
dwac_r_means_knn, dwac_r_confs_knn, dwac_r_means_svm, dwac_r_confs_svm = utils.get_random_score(data, k_range, m_range)
dwac_p_scores_knn, dwac_p_scores_svm = utils.get_protodash_score(data, k_range, m_range)

In [ ]:
fig, ax = plt.subplots(1, len(k_range), figsize=(16, 6), sharey=True)
for k in range(len(k_range)):
    ax[k].axhline(dwac_f_scores_knn[k], c='black')
    ax[k].plot(m_range, dwac_r_means_knn[k])
    ax[k].fill_between(m_range, dwac_r_means_knn[k] + dwac_r_confs_knn[k] / 2, dwac_r_means_knn[k] - dwac_r_confs_knn[k] / 2, alpha=0.5)
    ax[k].plot(m_range, dwac_p_scores_knn[k])
    ax[k].set_ylim(0.3, 1)
    ax[k].set_xticks(m_range)
    ax[k].set_title('K={}'.format(k_range[k]))
    ax[k].legend(['full', 'random', '', 'protodash', 'pdash_e', 'proto_g', 'protoclass', 'lpips'])
fig.suptitle('{}, {}.{}, AUC, KNN weights '.format(dataset, model, name) + "uniform")

In [ ]:
model= "tste"
train_path = '{}_train_px.pkl'.format(model)
valid_path = '{}_valid_px.pkl'.format(model)
f_train_tste, y_train_tste, x_train_tste = pickle.load(open(train_path, "rb"))
f_valid_tste, y_valid_tste, x_valid_tste = pickle.load(open(valid_path, "rb"))

data = x_train_tste, y_train_tste, x_valid_tste, y_valid_tste

In [ ]:
tste_f_scores_knn, tste_f_score_svm = utils.get_full_score(data, k_range)
tste_r_means_knn, tste_r_confs_knn, tste_r_means_svm, tste_r_confs_svm = utils.get_random_score(data, k_range, m_range)
tste_p_scores_knn, tste_p_scores_svm = utils.get_protodash_score(data, k_range, m_range)

In [ ]:
fig, ax = plt.subplots(1, len(k_range), figsize=(16, 6), sharey=True)
for k in range(len(k_range)):
    ax[k].axhline(tste_f_scores_knn[k], c='black')
    ax[k].plot(m_range, tste_r_means_knn[k])
    ax[k].fill_between(m_range, tste_r_means_knn[k] + tste_r_confs_knn[k] / 2, tste_r_means_knn[k] - tste_r_confs_knn[k] / 2, alpha=0.5)
    ax[k].plot(m_range, tste_p_scores_knn[k])
    ax[k].set_ylim(0.3, 1)
    ax[k].set_xticks(m_range)
    ax[k].set_title('K={}'.format(k_range[k]))
    ax[k].legend(['full', 'random', "", 'tste', 'pdash_e', 'proto_g', 'protoclass', 'lpips'])
fig.suptitle('{}, {}, AUC, KNN weights '.format(dataset, model) + "uniform")

In [ ]:
fig, ax = plt.subplots(1, len(k_range), figsize=(16, 6), sharey=True)
for k in range(len(k_range)):
    ax[k].plot(m_range, dwac_p_scores_knn[k])
    ax[k].plot(m_range, tste_p_scores_knn[k])
    ax[k].set_ylim(0.3, 1)
    ax[k].set_xticks(m_range)
    ax[k].set_title('K={}'.format(k_range[k]))
    ax[k].legend(['protodash', 'tste'])
fig.suptitle('{}, dwac vs tste, AUC, KNN weights uniform'.format(dataset))